NER import statement  

In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Zip import statments

In [ ]:
pip install wordfreq

In [ ]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.stem.lancaster import LancasterStemmer

Bert import statement  

In [ ]:
!pip install transformers
!pip install torch

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

bert_model = 'bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model)
model = BertForMaskedLM.from_pretrained(bert_model)
model.eval()

#CWI using ZIP


In [ ]:
stop_words_ = set(stopwords.words('english'))

# the word is turned to lower case and characters other than letters are removed
def cleaner(word):
  # Remove links
  # word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', 
  #               '', word, flags=re.MULTILINE)
  # word = re.sub('[\W]', ' ', word)
  word = re.sub('[^a-zA-Z]', ' ', word)           
  return word.lower().strip()

In [ ]:
from wordfreq import zipf_frequency

# Complex Word Identification
def get_list_cwi_predictions(input_text):
  list_cwi_predictions = []
  list_of_words=word_tokenize(input_text)
  for word in list_of_words:
    word = cleaner(word)
    if zipf_frequency(word, 'en') < 4:
      list_cwi_predictions.append(True)
    else:
      list_cwi_predictions.append(False)
  return list_cwi_predictions     

# BERT


In [ ]:
# basic Named Entity Recognition code

def NER_identifier(text):
  entity_list=[]
  nlp = en_core_web_sm.load()
  doc = nlp(text)
  for x in doc.ents:
    entity_tokens = tokenizer.tokenize(x.text)
    for word in entity_tokens:
      entity_list.append(word)
  return entity_list

In [ ]:
# BERT model to predict candidates for identified complex words
def get_bert_candidates(input_text, list_cwi_predictions, numb_predictions_displayed = 2):
  
  list_candidates_bert = []
  names_enitites=NER_identifier(input_text)
  for word,pred  in zip(input_text.split(), list_cwi_predictions):
    
    lowercase_word = word.lower()
    
    if pred and lowercase_word not in names_enitites :
      replace_word_mask = input_text.replace(word, '[MASK]')
      text = f'[CLS]{replace_word_mask} [SEP] {input_text} [SEP] '
      tokenized_text = tokenizer.tokenize(text)
      masked_index = [i for i, x in enumerate(tokenized_text) if x == '[MASK]'][0]
      indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
      segments_ids = [0]*len(tokenized_text)
      tokens_tensor = torch.tensor([indexed_tokens])
      segments_tensors = torch.tensor([segments_ids])
      
      # Predict all tokens
      with torch.no_grad():
          outputs = model(tokens_tensor, token_type_ids=segments_tensors)
          predictions = outputs[0][0][masked_index]
      predicted_ids = torch.argsort(predictions, descending=True)[:numb_predictions_displayed]
      predicted_tokens = tokenizer.convert_ids_to_tokens(list(predicted_ids))
      list_candidates_bert.append((word, predicted_tokens))

  return list_candidates_bert

# Run the code

In [ ]:
def remove_punctuation(input):
  x = re.sub("[^-9A-Za-z ]", "" , input)
  return x

In [ ]:
input_list = ['(5) Where an offence under subsection (1) is committed by a body corporate, any person who at the time of the commission of the offence was a director, general manager, secretary or other similar officer of the body corporate, or was purporting to act in any such capacity, shall be deemed to be guilty of that offence, unless he proves that the offence was committed without his consent or connivance and that he exercised all such diligence to prevent the commission of the offence as he ought to have exercised having regard to the nature of his functions in that capacity and to all the circumstances.'
 ]

In [ ]:
for input in input_list:
  input_no_punc=remove_punctuation(input)
  pred_list = get_list_cwi_predictions(input_no_punc)

  ps = LancasterStemmer() 
 
  candidates = get_bert_candidates(input_no_punc, pred_list)
  
  for word,pred  in zip(input_no_punc.split(), pred_list):
    if pred == True:
      for candidate in candidates:
        if candidate[0] == word:
          replacement = ""
          if zipf_frequency(candidate[1][0], 'en') > zipf_frequency(candidate[1][1], 'en'):
            replacement = candidate[1][0]
          else:
            replacement = candidate[1][1]
          
          if ps.stem(replacement) == ps.stem(word) or (not replacement.isalpha()): 
            replacement = word
          input = input.replace(word, replacement)
  print('Output - ', input)


Output -  (5) Where an offence under subsection (1) is committed by a body corporate, any person who at the time of the commission of the offence was a director, general manager, secretary or other similar officer of the body corporate, or was authorised to act in any such capacity, shall be deemed to be guilty of that offence, unless he proves that the offence was committed without his consent or knowledge and that he exercised all such power to prevent the commission of the offence as he ought to have exercised having regard to the nature of his functions in that capacity and to all the circumstances.
